In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import polars as pl
import matplotlib.pyplot as plt

import stock

In [ ]:
# trend template策定のために値上がり銘柄を探索する
csv_dir = stock.PROJECT_ROOT / "data" / "daily"

In [ ]:
# 2023/10月くらいからの上昇相場で大きく値上がりした銘柄のリストを取得
min_rise_rate = 2.0
start_date = datetime(year=2023, month=7, day=1)
rise_rates = {}

for csv_path in sorted(csv_dir.glob("*.csv")):
    df = stock.kabutan.read_data_csv(csv_path, exclude_none=True)
    df = df.filter(pl.col("date") >= start_date)
    
    min_idx, max_idx = df["close"].arg_min(), df["close"].arg_max()
    if min_idx is None or max_idx is None or min_idx > max_idx:
        continue
    rise_rates[csv_path.stem]  = df["close"][max_idx]  / df["close"][min_idx]

In [ ]:
thresh = 3.0
targets = {code: val for code, val in rise_rates.items() if val > thresh}

In [ ]:
def plot_with_rs(df, start_date, end_date=datetime.today()):
    fig = plt.figure()
    df = df.filter((pl.col("date") >= start_date) & (pl.col("date") <= end_date))
    ax1 = fig.add_subplot(111)
    ax1.plot(df["date"], df["close"], "red", label="close")

    ax2 = ax1.twinx()
    ax2.plot(df["date"], df["rs_topix"], "blue", label="rs_topix")
    ax1.grid(True)
    ax1.tick_params(axis='x', labelrotation=90)

    plt.legend()

In [ ]:
# 成長銘柄
selected = {
    "2160": datetime(year=2023, month=11, day=14),
    "2708": datetime(year=2023, month=8, day=10),
    "2820": datetime(year=2023, month=8, day=28),
    "3399": datetime(year=2023, month=3, day=12),
    "3498": datetime(year=2023, month=9, day=3),
    "3778": datetime(year=2023, month=6, day=12),
    "3865": datetime(year=2023, month=9, day=1),
    "6016": datetime(year=2023, month=4, day=3),
    "6254": datetime(year=2023, month=10, day=24),
    "7003": datetime(year=2024, month=2, day=7),
    "7014": datetime(year=2023, month=8, day=8),
    "9268": datetime(year=2023, month=7, day=19),
}

In [ ]:
sorted(selected.keys())[0]

In [ ]:
# 成長銘柄のrsと株価の動きを調査
code = sorted(selected.keys())[5]
csv_path = csv_dir / f"{code}.csv"
df = stock.kabutan.read_data_csv(csv_path)
start_date = selected[code] - timedelta(days=90)  # 注目1ヶ月前からプロットする
plot_with_rs(df, start_date)

In [ ]:
params = stock.trend_template.TechnicalTrendTemplateParams(
    high_ma_weeks=[10, 30, 40],
    uptrend_ma_weeks = [40], # 週移動平均線が上向きか
    uptrend_days = 20,  # 週移動平均線が上向きかチェックする日数
    near_high_week = 52,  # 高値に近いか
    max_rate_from_high = 0.25,  # 高値からの最大割合
    min_rate_from_low = 0.3,  # 安値からの最小割合
)

In [ ]:
code = "9268"
start_date = datetime(year=2023, month=1, day=1)
stock.trend_template.calc_watch_list_duration_of(code, start_date=start_date)

In [ ]:
stock.trend_template.calc_watch_list_duration_of(code, start_date=start_date, use_fundamental=False)

In [ ]:
results = {}
stock.trend_template.check_fundamental_trend_templates(code, datetime(year=2023, month=4, day=15), results)
print(results)

In [ ]:
target_date = datetime(year=2023, month=8, day=10)
ttt = stock.trend_template.TechnicalTrendTemplate(code)
ttt.check(day=target_date)
ttt.get_debug_info()